# 📊 Analyse Exploratoire des Données (EDA)

Ce notebook explore les données MMM pour comprendre :
- Les distributions des variables
- Les corrélations entre dépenses media et ventes
- Les tendances et la saisonnalité
- Les patterns de dépenses

**Auteur** : Ivan  
**Projet** : MMM Bayésien - MSMIN5IN43

In [ ]:
# Imports
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data.loader import load_csv_data, validate_mmm_data, get_dataset_summary
from data.preprocessing import handle_missing_values, add_time_features
from visualization.exploratory import (
    plot_time_series,
    plot_sales_vs_media,
    plot_correlation_matrix,
    plot_distributions,
    plot_seasonality_decomposition,
    plot_spending_patterns,
    create_eda_report
)

# Config plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
%matplotlib inline

# Pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

## 1️⃣ Chargement des données

In [ ]:
# Charger le dataset
df = load_csv_data('../data/raw/sample_data.csv')

print(f"✓ Dataset chargé : {len(df)} périodes")
print(f"✓ Colonnes : {list(df.columns)}")
print(f"\n📅 Période : {df['date'].min()} → {df['date'].max()}")

df.head(10)

## 2️⃣ Statistiques descriptives

In [ ]:
# Stats descriptives
print("📊 Statistiques descriptives\n")
print(df.describe())

In [ ]:
# Résumé complet du dataset
summary = get_dataset_summary(df)

print("\n📈 Résumé du dataset")
print(f"Nombre de périodes : {summary['n_periods']}")
print(f"Nombre de colonnes : {summary['n_columns']}")
print(f"Plage de dates : {summary['date_range']}")

print("\n💰 Statistiques des ventes :")
sales_stats = summary['numeric_stats']['sales']
print(f"  Moyenne : {sales_stats['mean']:.1f}")
print(f"  Écart-type : {sales_stats['std']:.1f}")
print(f"  Min : {sales_stats['min']:.1f}")
print(f"  Max : {sales_stats['max']:.1f}")

## 3️⃣ Validation du dataset

In [ ]:
# Valider le dataset
media_columns = ['media_1_spend', 'media_2_spend', 'media_3_spend']
report = validate_mmm_data(df, media_columns=media_columns)

print("✅ Validation du dataset\n")
print(f"Dataset valide : {report['valid']}")
print(f"Nombre de périodes : {report['n_periods']}")
print(f"Canaux media : {report['n_media_channels']}")

if report['warnings']:
    print("\n⚠️  Warnings :")
    for w in report['warnings']:
        print(f"  - {w}")

if report['errors']:
    print("\n❌ Erreurs :")
    for e in report['errors']:
        print(f"  - {e}")

## 4️⃣ Visualisations

### 4.1 Séries temporelles

In [ ]:
# Visualiser toutes les séries
all_cols = ['sales'] + media_columns
fig = plot_time_series(df, all_cols, title='Évolution temporelle des ventes et dépenses media')
plt.show()

### 4.2 Relations Ventes vs Media

In [ ]:
# Scatter plots avec corrélations
fig = plot_sales_vs_media(df, target_column='sales', media_columns=media_columns)
plt.show()

### 4.3 Matrice de corrélation

In [ ]:
# Heatmap des corrélations
fig = plot_correlation_matrix(df, columns=all_cols)
plt.show()

In [ ]:
# Table des corrélations avec les ventes
print("\n📊 Corrélations avec les ventes\n")
corr_with_sales = df[all_cols].corr()['sales'].sort_values(ascending=False)
print(corr_with_sales)

### 4.4 Distributions

In [ ]:
# Histogrammes et densités
fig = plot_distributions(df, all_cols)
plt.show()

### 4.5 Décomposition saisonnalité

In [ ]:
# Décomposer les ventes (tendance + saisonnalité + résidus)
fig = plot_seasonality_decomposition(df, 'sales', period=52)
plt.show()

### 4.6 Patterns de dépenses media

In [ ]:
# Stacked area des dépenses
fig = plot_spending_patterns(df, media_columns)
plt.show()

## 5️⃣ Insights clés

### 📌 Observations principales

In [ ]:
# Calculer quelques métriques clés
print("🔍 Insights clés\n")

# Total dépenses par canal
print("💰 Dépenses totales par canal :")
for col in media_columns:
    total = df[col].sum()
    pct = (total / df[media_columns].sum().sum()) * 100
    print(f"  {col:20s} : {total:10,.0f} ({pct:.1f}%)")

print(f"\n  Total : {df[media_columns].sum().sum():,.0f}")

# Ventes totales et moyennes
print(f"\n📊 Ventes :")
print(f"  Total : {df['sales'].sum():,.0f}")
print(f"  Moyenne/période : {df['sales'].mean():,.1f}")
print(f"  Croissance : {((df['sales'].iloc[-1] / df['sales'].iloc[0]) - 1) * 100:.1f}%")

# ROI approximatif par canal (très simplifié)
print(f"\n💡 ROI approximatif (corrélation simple) :")
for col in media_columns:
    corr = df[[col, 'sales']].corr().iloc[0, 1]
    print(f"  {col:20s} : {corr:.3f}")

## 6️⃣ Préparation pour la modélisation

In [ ]:
# Ajouter les features temporelles
df_enriched = add_time_features(df)

print("✓ Features temporelles ajoutées :")
time_features = ['trend', 'seasonality_sin', 'seasonality_cos', 'month', 'quarter']
print(f"  {time_features}")

print(f"\n📊 Nouvelles colonnes : {len(df_enriched.columns)} (avant : {len(df.columns)})")
df_enriched.head()

## 7️⃣ Conclusions

### ✅ Résumé de l'EDA

**Dataset :**
- ✓ 104 périodes (2 ans hebdomadaires)
- ✓ 3 canaux media
- ✓ Pas de valeurs manquantes

**Patterns identifiés :**
- Tendance croissante des ventes
- Saisonnalité annuelle visible
- Corrélations positives entre media et ventes

**Prochaines étapes :**
1. Modèle MMM simple (régression linéaire)
2. Ajout des transformations (adstock + saturation)
3. Modèle bayésien complet avec PyMC
4. Attribution et optimisation budget